## Local Inference

In [1]:
%matplotlib inline
import cv2, numpy as np, matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.models.detection import KeypointRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F
import time
import tensorflow as tf

%matplotlib inline

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [7]:
def get_model(num_keypoints, weights_path=None):
    
    anchor_generator = AnchorGenerator(sizes=(32, 64, 128, 256, 512), aspect_ratios=(0.25, 0.5, 0.75, 1.0, 2.0, 3.0, 4.0))
    model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=False,
                                                                   pretrained_backbone=True,
                                                                   num_keypoints=num_keypoints,
                                                                   num_classes = 2, # Background is the first class, object is the second class
                                                                   rpn_anchor_generator=anchor_generator)

    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)        
        
    return model

In [8]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = get_model(num_keypoints = 4, weights_path='./assets/keypoint_model/weights/keypointsrcnn_weights 001.pth')
model.to(device)

c:\Users\hiski\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hiski\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
c:\Users\hiski\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained_backbone' is deprecated since 0.13 and may be removed in the future, please use 'weights_backbone' instead.
  warnings.warn(
c:\Users\hiski\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum

KeypointRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(640, 672, 704, 736, 768, 800), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, 

### Live Frame filter Algorithm

In [9]:
def calculate_bbox_center(bbox):
    # calculate center of bbox
    bbox_width = abs(bbox[0] - bbox[2])
    bbox_height = abs(bbox[1] - bbox[3])

    return [bbox[0] + bbox_width / 2, bbox[1] + bbox_height / 2], bbox_width, bbox_height


In [10]:
def live_frame_filter(keypoint, bbox):
    filtered_keypoints = []
    filtered_frames = []
    # filter threshold
    x_threshold = 0.5
    y_threshold = 6

    # calculate center of bbox
    bbox_center, bbox_width, bbox_height = calculate_bbox_center(bbox)

    # find left and right side coordinates
    left_side_coordinates = []
    right_side_coordinates = []

    # Get the left and right side coordinates
    for point in keypoint:
        if bbox_center[0] < point[0]:
            left_side_coordinates.append(point)
        elif bbox_center[0] > point[0]:
            right_side_coordinates.append(point)

    if len(left_side_coordinates) != 2 or len(right_side_coordinates) != 2:
        return False

    # Get the top and bottom side coordinates
    top_side_coordinates = []
    bottom_side_coordinates = []

    for point in keypoint:
        if bbox_center[1] < point[1]:
            top_side_coordinates.append(point)
        elif bbox_center[1] > point[1]:
            bottom_side_coordinates.append(point)
    
    if len(top_side_coordinates) != 2 or len(bottom_side_coordinates) != 2:
        return False

    # Extract the slope angles
    left_side_slope = 0 if left_side_coordinates[0][0] - left_side_coordinates[1][0] == 0 else (left_side_coordinates[0][1] - left_side_coordinates[1][1]) / (left_side_coordinates[0][0] - left_side_coordinates[1][0])
    right_side_slope = 0 if right_side_coordinates[0][0] - right_side_coordinates[1][0] == 0 else (right_side_coordinates[0][1] - right_side_coordinates[1][1]) / (right_side_coordinates[0][0] - right_side_coordinates[1][0])
    top_side_slope = 0 if top_side_coordinates[1][0] - top_side_coordinates[0][0] == 0 else (top_side_coordinates[1][1] - top_side_coordinates[0][1]) / (top_side_coordinates[1][0] - top_side_coordinates[0][0])
    bottom_side_slope = 0 if bottom_side_coordinates[1][0] - bottom_side_coordinates[0][0] == 0 else (bottom_side_coordinates[1][1] - bottom_side_coordinates[0][1]) / (bottom_side_coordinates[1][0] - bottom_side_coordinates[0][0])

    diff_top_bottom = abs(top_side_slope - bottom_side_slope)
    diff_left_right = abs(left_side_slope - right_side_slope)

    if diff_top_bottom < x_threshold and diff_left_right < y_threshold:
        return True
        
    return False

## Run model on Live Camera or Video Recording

In [11]:
# frame capture rate in seconds
capture_rate = 5

# video file
video_file = "./assets/pfd_video_dataset/20221226_133619.mp4"

### Test on video recording 
Choose from:
* Display video and draw over bounding box
* Extract frames for further processing (saves more time)

In [12]:
# with displaying video
frame_list = []
keypoint_list = []
bbox_list = []

# Open the video file
video = cv2.VideoCapture(video_file)

# Check if video is opened successfully
if not video.isOpened():
    print("Error opening video file")

# Get the width and height of the video
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set the desired width
desired_width = 500

# Calculate the aspect ratio
aspect_ratio = height / width

# Calculate the new height
desired_height = int(desired_width * aspect_ratio)

# Get the frames per second of the video
fps = video.get(cv2.CAP_PROP_FPS)

# Calculate the number of frames to skip
skip_frames = int(fps * capture_rate)

#### Display video and draw keypoints and bounding box

In [ ]:
# Initialize the counter for frames
frame_count = 0

# keypoint, bbox and frame lists 
keypoints_main = []
bboxes_main = []

# Read until video is completed
while(video.isOpened()):
    # Capture frame-by-frame
    ret, frame = video.read()
    
    if not ret:
        break
    
    if frame_count % skip_frames == 0:
        frame_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)            
        frame_tensor = F.to_tensor(frame_np)
        frame_tensor = frame_tensor.to(device)
        
        images = [frame_tensor]

        with torch.no_grad():
            model.to(device)
            model.eval()
            output = model(images)
        
        frame_tensor = (images[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
        scores = output[0]['scores'].detach().cpu().numpy()
        
        high_scores_idxs = np.where(scores > 0.8)[0].tolist() # Indexes of boxes with scores > 0.7
        post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], output[0]['scores'][high_scores_idxs], 0.3).cpu().numpy() # Indexes of boxes left after applying NMS (iou_threshold=0.3)

        keypoints = []
        for kps in output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            keypoints.append([list(map(int, kp[:2])) for kp in kps])
            
        bboxes = []
        for bbox in output[0]['boxes'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            bboxes.append(list(map(int, bbox.tolist())))

        #filter keypoints
        if len(keypoints) != 0 and live_frame_filter(keypoints[0], bboxes[0]):
            if(len(keypoints) != 0):
                keypoints_main = keypoints
            
            if(len(bboxes) != 0):
                bboxes_main = bboxes

        # add frame to to frame_list
        frame_list.append(cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2RGB))
        keypoint_list.append(keypoints_main)
        bbox_list.append(bboxes_main)

    if(len(keypoints_main) != 0):
        points = np.array(keypoints_main[0], np.int32)
        points = points.reshape((-1, 1, 2))

        # Draw a keypoints on the frame
        # cv2.polylines(frame, [points], isClosed=True, color=(0, 0, 0), thickness=8)
        for point in points:
            center = (point[0][0], point[0][1])
            radius = 15

            # Draw the keypoints using the circle() function
            cv2.circle(frame, center, radius, (0, 0, 255), -1)

    if(len(bboxes_main) != 0):
        # draw a bbox on the frame
        cv2.rectangle(frame, (bboxes_main[0][0], bboxes_main[0][1]),  (bboxes_main[0][2], bboxes_main[0][3]), color=(0, 255, 0), thickness=8)

    # Display the resulting frame
    frame = cv2.resize(frame, (desired_width, desired_height), interpolation=cv2.INTER_AREA)
    cv2.imshow('Frame', frame)
    
    # Press Q on keyboard to exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    frame_count = frame_count + 1
# Release the video
video.release()

# Close all windows
cv2.destroyAllWindows()

#### Extract frames and keypoints for further processing

In [13]:
# Get the total number of frames in the video
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(video.get(cv2.CAP_PROP_FPS))

# keypoint, bbox and frame lists 
keypoints_main = []
bboxes_main = []

# Loop through the frames of the video
for i in range(total_frames):
    # Capture frame-by-frame
    ret, frame = video.read()

    if i % skip_frames == 0:
        frame_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)            
        frame_tensor = F.to_tensor(frame_np)
        frame_tensor = frame_tensor.to(device)

        images = [frame_tensor]
        
        with torch.no_grad():
            model.to(device)
            model.eval()
            output = model(images)

        frame_tensor = (images[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
        scores = output[0]['scores'].detach().cpu().numpy()

        high_scores_idxs = np.where(scores > 0.8)[0].tolist() # Indexes of boxes with scores > 0.7
        post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], output[0]['scores'][high_scores_idxs], 0.3).cpu().numpy() # Indexes of boxes left after applying NMS (iou_threshold=0.3)

        keypoints = []
        for kps in output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            keypoints.append([list(map(int, kp[:2])) for kp in kps])
            
        bboxes = []
        for bbox in output[0]['boxes'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            bboxes.append(list(map(int, bbox.tolist())))

        #filter keypoints
        if len(keypoints) != 0 and live_frame_filter(keypoints[0], bboxes[0]):
            if(len(keypoints) != 0):
                keypoints_main = keypoints
            
            if(len(bboxes) != 0):
                bboxes_main = bboxes
        
        # add frame to to frame_list
        if len(keypoints_main) > 0:
            frame_list.append(frame_tensor)
            keypoint_list.append([keypoints_main[0]])
            bbox_list.append(bboxes)

# Release the video
video.release()

### Test on live camera

In [4]:
# Initialize the counter for frames
frame_count = 0

# keypoint, bbox and frame lists 
keypoints_main = []
bboxes_main = []

# Create a VideoCapture object
cap = cv2.VideoCapture(0) # 0 means the default camera

while True:
    # Read a frame from the video
    ret, frame = cap.read()

    if int(time.time()) % capture_rate == 0:
        frame_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_tensor = F.to_tensor(frame_np)
        frame_tensor = frame_tensor.to(device)
        images = [frame_tensor]


        with torch.no_grad():
            model.to(device)
            model.eval()
            output = model(images)
        
        frame_tensor = (images[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
        scores = output[0]['scores'].detach().cpu().numpy()
        
        high_scores_idxs = np.where(scores > 0.7)[0].tolist() # Indexes of boxes with scores > 0.7
        post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], output[0]['scores'][high_scores_idxs], 0.3).cpu().numpy() # Indexes of boxes left after applying NMS (iou_threshold=0.3)

        keypoints = []
        for kps in output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            keypoints.append([list(map(int, kp[:2])) for kp in kps])
            
        bboxes = []
        for bbox in output[0]['boxes'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            bboxes.append(list(map(int, bbox.tolist())))
    
        if(len(keypoints) != 0):
            print(keypoints)
            keypoints_main = keypoints
        
        if(len(bboxes) != 0):
            print(bboxes)
            bboxes_main = bboxes

        # add frame to to frame_list
        frame_list.append(frame)
        keypoint_list.append(keypoints_main)
        bbox_list.append(bboxes_main)

    if(len(keypoints_main) != 0):
        points = np.array(keypoints_main, np.int32)
        points = points.reshape((-1, 1, 2))

        # Draw a keypoints on the frame
        # cv2.polylines(frame, [points], isClosed=True, color=(0, 0, 0), thickness=8)
        for point in points:
            center = (point[0][0], point[0][1])
            radius = 15

            # Draw the keypoints using the circle() function
            cv2.circle(frame, center, radius, (255, 0, 0), -1)

    if(len(bboxes_main) != 0):
        # draw a bbox on the frame
        cv2.rectangle(frame, (bboxes_main[0][0], bboxes_main[0][1]),  (bboxes_main[0][2], bboxes_main[0][3]), color=(0, 255, 0), thickness=8)
        
    # Display the frame with the bounding box
    cv2.imshow("Live Video with Bounding Box", frame)

    # `Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count = frame_count + 1

# Release the video capture object
cap.release()

# Close all windows
cv2.destroyAllWindows()

## Perspective Transformation and Timelapse

#### Perspective Transformation and Timelapse Creation

In [464]:
# Calculate the frame rate
num_frames = len(frame_list)
timelapse_length = num_frames / 2
frame_rate = int(num_frames / timelapse_length)

#

# previous frame for blending
prev_frame = []
keypoint = keypoint_list[0][0]
bbox = bbox_list[0][0]

bbox_center, bbox_width, bbox_height = calculate_bbox_center(bbox)

#  Define the codec and create a video writer
fourcc = cv2.VideoWriter_fourcc(*"MP4V" )
out = cv2.VideoWriter("timelapse.mp4", fourcc, frame_rate, (bbox_width, bbox_height))

for count, frame in enumerate(frame_list):
    # pt1 => (top_left, top_right, bottom_left, bottom_right)
    top_left = []
    top_right = []
    bottom_left = []
    bottom_right = []

    for kp in keypoint:
        if kp[0] < bbox_center[0] and kp[1] < bbox_center[1]:
            top_left = kp
        elif kp[0] > bbox_center[0] and kp[1] > bbox_center[1]:
            bottom_right = kp
        elif kp[0] < bbox_center[0] and kp[1] > bbox_center[1]:
            top_right = kp
        elif kp[0] > bbox_center[0] and kp[1] < bbox_center[1]:
            bottom_left = kp

    pt1 = np.float32([top_left, bottom_left, top_right, bottom_right])
    pt2 = np.float32([[0,0],[bbox_width, 0], [0, bbox_height], [bbox_width, bbox_height]])
    M = cv2.getPerspectiveTransform(pt1,pt2)

    # Perform the transformation
    warped_image = cv2.warpPerspective(frame, M, (bbox_width, bbox_height))
    warped_image = cv2.cvtColor(warped_image,cv2.COLOR_BGR2RGB)

    # blend frames together for better transition
    if count != 0:
        blended = cv2.addWeighted(prev_frame, 0.1, warped_image, 0.9, 0)
    else:
        blended = warped_image
    prev_frame = warped_image

    out.write(blended)

# Release the video writer
out.release()

## Export Model for Android

In [50]:
from torch.utils.mobile_optimizer import optimize_for_mobile

model = get_model(num_keypoints = 4, weights_path='./assets/keypoint_model/weights/keypointsrcnn_weights 001.pth')

model.eval()

KeypointRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(640, 672, 704, 736, 768, 800), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, 

In [14]:
x = [F.to_tensor(frame_list[0]), F.to_tensor(frame_list[1])]

In [52]:
predictions = model(x)

In [58]:
# optionally, if you want to export the model to ONNX:
torch.onnx.export(model, x, "./assets/keypoint_rcnn.onnx", opset_version = 11)

In [15]:
import onnx

onnx_model = onnx.load('./assets/keypoint_rcnn.onnx')

In [17]:
import onnxruntime as onnxrt

onnx_session= onnxrt.InferenceSession("./assets/keypoint_rcnn.onnx", providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'],)
onnx_inputs= {onnx_session.get_inputs()[0].name: x[0]}

RuntimeError: D:\a\_work\1\s\onnxruntime\core\session\provider_bridge_ort.cc:1103 onnxruntime::ProviderLibrary::Get [ONNXRuntimeError] : 1 : FAIL : LoadLibrary failed with error 126 "" when trying to load "c:\Users\hiski\AppData\Local\Programs\Python\Python310\lib\site-packages\onnxruntime\capi\onnxruntime_providers_tensorrt.dll"


In [ ]:
from onnx_tf.backend import prepare

tf_rep = prepare(onnx_model, strict=False, gen_tensor_dict=True)

In [64]:
# Input nodes to the model
print('inputs:', tf_rep.inputs)

# Output nodes from the model
print('outputs:', tf_rep.outputs)

# All nodes in the model
print('tensor_dict:')
print(tf_rep.tensor_dict)

inputs: ['image.1', 'image.5']
outputs: ['3983', '3303', '3302', '3992', 'end_scores.7', '4013', '3424', '3423', '4022', 'end_scores.21']
tensor_dict:
{}


In [ ]:
tf_rep.run(x[0])

In [ ]:
tf_rep.export_graph('./assets/keypoint_tf/keypoint_tflite.pb')

In [22]:
script_model = torch.jit.script(model)
traced_script_module_optimized = optimize_for_mobile(script_model)

In [ ]:
traced_script_module_optimized._save_for_lite_interpreter("assets/keypoint_model/model_2.ptl")

In [14]:
torch.jit.save(traced_script_module_optimized, "assets/keypoint_model/android_model.ptl")